# Laboratorio 8: Percepción Remota Aplicada

## Álvaro Paredes (alvaro.paredes@dataobservatory.net) | Javier Lopatin (javier.lopatin@uai.cl)

En este práctico vamos a ver cómo utilizar Jupyter Notebooks con Kernel de Python para extraer datos en puntos de interés.

Primero, instalaremos las librerias correspondientes:



In [ ]:
%%capture
# Installations
!apt install gdal-bin python-gdal python3-gdal 
!pip install geopandas patool
!pip install xarray rasterio rioxarray xarray-spatial rasterstats

In [ ]:
# cargamos las librerias en el sistema
import xarray as xr
import rioxarray as riox
import xrspatial as xrs
import pandas as pd
import geopandas as gpd
import glob
import os
import rasterstats as rstats

# función para cargar un raster a la memoria del sistema
def rasterio_open(f):
    return riox.open_rasterio(f)

In [ ]:
# descargamos los datos necesarios para el laboratorio. La dirección de descarga puede ser cambiada
# alternativamente, pueden descargar y extraer los datos de forma manual en el PC
!git clone https://github.com/alvaroparedesl/percepcion_remota

Cloning into 'percepcion_remota'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 124 (delta 3), reused 16 (delta 3), pack-reused 108
Receiving objects: 100% (124/124), 798.55 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Checking out files: 100% (83/83), done.


## Cargar Datos

In [ ]:
dem = rasterio_open('percepcion_remota/Data/Lab08/DEM/cop_30_dem.tif').squeeze(drop=True)
dem

<xarray.DataArray (y: 2043, x: 1337)>
[2731491 values with dtype=float32]
Coordinates:
  * x            (x) float64 1.643e+05 1.643e+05 ... 2.043e+05 2.043e+05
  * y            (y) float64 6.038e+06 6.038e+06 ... 5.977e+06 5.977e+06
    spatial_ref  int64 0
Attributes:
    _FillValue:    -32767.0
    scale_factor:  1.0
    add_offset:    0.0

In [ ]:
bands_names = glob.glob('percepcion_remota/Data/Lab08/L8/*.tif')
images = []

for band in bands_names:
  bandsi = rasterio_open(band)
  bandsi.name = os.path.basename(band).split("_")[-1].split('.')[0]  # asignar el nombre de la banda al DataArray
  # Estas dos lineas que siguen, permiten agregar la fecha como una coordenada
  bandsi.coords['time'] = ('band', [pd.to_datetime(os.path.basename(band).split('_')[3])])
  bandsi = bandsi.swap_dims({'band': 'time'}).drop('band')
  images.append(bandsi)

L8 = xr.merge(images)
L8

<xarray.Dataset>
Dimensions:      (time: 7, x: 1337, y: 2043)
Coordinates:
  * time         (time) datetime64[ns] 2021-02-20 2021-03-08 ... 2021-12-05
  * x            (x) float64 1.643e+05 1.643e+05 ... 2.043e+05 2.043e+05
  * y            (y) float64 6.038e+06 6.038e+06 ... 5.977e+06 5.977e+06
    spatial_ref  int64 0
Data variables:
    nir08        (time, y, x) float64 7.307e+03 7.356e+03 ... 1.586e+04
    green        (time, y, x) float64 7.54e+03 7.594e+03 ... 9.647e+03 9.371e+03
    swir22       (time, y, x) float64 7.479e+03 7.511e+03 ... 1.211e+04
    swir16       (time, y, x) float64 7.491e+03 7.545e+03 ... 1.616e+04
    coastal      (time, y, x) float64 7.27e+03 7.315e+03 ... 8.11e+03 8.058e+03
    red          (time, y, x) float64 7.302e+03 7.349e+03 ... 1.023e+04
    blue         (time, y, x) float64 7.377e+03 7.399e+03 ... 8.408e+03
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0

In [ ]:
df = pd.read_csv('percepcion_remota/Data/Lab08/Parcelas BN y riqueza.csv')
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(x=df.X, y=df.Y), crs=32718
)
gdf.set_index('Parcela', inplace=True)
gdf.head()

,X,Y,Huso,Riqueza observada,geometry
Parcela,,,,,
BMBNDIE1,725584.0,6009648.0,UTM 18S,15,POINT (725584.000 6009648.000)
BMBNDIE2,725661.0,6010192.0,UTM 18S,8,POINT (725661.000 6010192.000)
BMBNDIE3,725515.0,6010219.0,UTM 18S,9,POINT (725515.000 6010219.000)
BMBNDIE5,725250.0,6010317.0,UTM 18S,5,POINT (725250.000 6010317.000)
BMBNMIR2,727241.0,6013933.0,UTM 18S,6,POINT (727241.000 6013933.000)


# Actividades

1. DEM:
  1. Obtenga la pendiente
  1. Obtenga la orientación
2. Landsat 8
  1. Obtenga la serie temporal de NDVI
3. Para cada escena de NDVI, extraiga el valor del pixel de cada parcela. Exporte a un archivo csv
4. Para cada escena de NDVI, extraiga el valor promedio, máximo y mínimo alrededor de cada parcela, usando un buffer de 60 metros.
5. Repita, utilizando los valores del DEM para elevación, pendiente y orientación
6. Consolide la base de datos en un único archivo

RECUERDE:
  1. Revisar que todo esté en el mismo sistema de coordenadas (idealmente proyectado)
  1. En el caso de las imágenes, si provienen de diferentes sensores, recuerde alinearlas.

Funciones útiles:
  1. Sobre `xarray` (raster).
    1. `xarray.where` (`xr.where`): genera máscaras de manera más sencilla
    1. `xrspatial.aspect`: cálculo de orientación sobre un DEM
    1. `xrspatial.slope`: cálculo de la pendiente sobre un DEM
  1. Sobre `geopandas` (vector):
    1. `gdf.buffer`: genera un buffer sobre la geometría
  1. Sobre ambas:
    1. `rstats.zonal_stats`: estadísticas de zona (resumen) para polígonos sobre un raster

# Ejemplos


In [ ]:
x_ = xr.DataArray([185260, 179752])
y_ = xr.DataArray([6008515, 6021478])
valores = L8.isel(time=0).sel(x=x_, y=y_, method="nearest").to_array().values
pd.DataFrame(valores, index=L8.keys())

,0,1
nir08,16842.0,13596.0
green,8057.0,7802.0
swir22,8276.0,8197.0
swir16,9821.0,9516.0
coastal,7332.0,7167.0
red,7822.0,7648.0
blue,7450.0,7305.0


# Solución

In [ ]:
gdfr = gdf.to_crs(32719)

In [ ]:
vals = rstats.zonal_stats(gdfr.buffer(60).geometry,
                          L8.isel(time=0)['nir08'].values,
                          affine=L8.rio.transform(),
                          nodata=-999,
                          all_touched=True,
                          stats=['min', 'max', 'median', 'majority', 'sum', 'count', 'std'])

In [ ]:
pd.DataFrame(vals, index=gdfr.index)

,min,max,count,sum,std,median,majority
Parcela,,,,,,,
BMBNDIE1,12904.0,16555.0,22,325969.0,1202.940977,14696.0,12904.0
BMBNDIE2,15663.0,17685.0,22,362196.0,564.568599,16358.5,15663.0
BMBNDIE3,13749.0,17127.0,21,326525.0,937.650134,15604.0,13749.0
BMBNDIE5,14019.0,17765.0,23,378339.0,891.687110,16543.0,14019.0
BMBNMIR2,13960.0,16562.0,22,333290.0,688.465403,15165.0,13960.0
...,...,...,...,...,...,...,...
SANESTEBAN3,12955.0,16844.0,22,330237.0,1232.577526,15097.5,12955.0
SANESTEBAN4,11724.0,14851.0,22,297831.0,944.364189,13769.0,13769.0
Turingia_BN_01,14417.0,15632.0,21,316398.0,330.345789,15076.0,14417.0
